In [2]:
# Importing requird python packages
# Path to hsmpy package
HSMPY_PATH = r'\\CHCFPP01\Guest\MRajabi\Python Scripts'
import sys, os, arcpy, csv, json, math
sys.path.append(HSMPY_PATH)
import hsmpy3
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import timedelta
from IPython.display import Image
from IPython.core.display import HTML 
import subprocess 
import webbrowser
import time
import copy
import shutil
from time import gmtime, strftime
print(sys.version)
print(strftime("%Y-%m-%d %H:%M:%S"))

3.5.3 |Continuum Analytics, Inc.| (default, May 15 2017, 10:43:23) [MSC v.1900 64 bit (AMD64)]
2019-02-07 10:00:41


In [3]:
# Defining Parameteres
print(strftime("%Y-%m-%d %H:%M:%S"))
Years = range(2005,2017)
MaxOpenProcesses = 15

Project_DIR  = r'\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO19\HSIP_Tracking_Tool'
Project_GDB  = Project_DIR + '\\2.Project_GDB.gdb'
ContractsDir = Project_DIR + '\\4.Contracts_AnalysisFiles'
DocumentsDir = Project_DIR + '\\5.Documents'
PythonDir    = Project_DIR + '\\6.Python_Codes'
ReceivedDir  = Project_DIR + '\\7.ReceivedData'
XML_Dir      = ReceivedDir + '\\HSIP Candidate Applications'


# IRIS Dataset
IRISPath = r'\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\7_BaseData\Master_BaseFiles_Data\5.0_Roadway_Layer\IRIS'
IRISPath = r'C:\Users\mr068144\Downloads\IL Data\IRIS\IL_IRIS.gdb'
IRIS_Route = {y:os.path.join(IRISPath,'HWY'+str(y)+'_route') for y in Years}
IRIS_Tab   = {y:os.path.join(IRISPath,'HWY'+str(y)+'_table_HCurve') for y in Years}
IRIS_Tab[2012] = IRIS_Tab[2013]
IRIS_Cleaned = r'\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO19\IL_Crash_Assignment\IRIS_Cleaned\IRIS_05_17.csv'

# Intersections Dataset
IntPG = r'C:\Users\mr068144\Downloads\IL Data\6.0_Intersection_Layer\IRIS_Ints.gdb\Int2014_PG'
Intersections = {year:IntPG for year in Years}

#Crash Dataset
CrashData  = {y:os.path.abspath(r'C:\Users\mr068144\Downloads\IL Data\IL Crash Data\IL_Crash.gdb\\' +
                          'CrashExtract_' + str(y) + '_GIS') for y in Years}
Crash_Cleaned = r'\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO19\IL_Crash_Assignment\CrashData_Cleaned\Crash_05_16.csv'

Fields = ['AADT',
          'O_SHD1_TYP','O_SHD1_WTH','O_SHD2_TYP','O_SHD2_WTH',
          'I_SHD1_TYP','I_SHD1_WTH','I_SHD2_TYP','I_SHD2_WTH',
          'LNS','LN_WTH','MED_TYP','MED_WTH','SURF_WTH','URBAN','PG','Radius','CurveLen']
#Fields = ['AADT','URBAN','PG']
RouteID = 'INVENTORY'
BMP = 'BEG_STA'
EMP = 'END_STA'

# Geocoded projects
Const_Int  = Project_GDB + '\\Const_Int_All'
Const_Seg  = Project_GDB + '\\Const_Seg_All'

HSIP_Path    = ReceivedDir + '\\HSIP_AllApprovedProjects.xlsm'
WPPS_Ob_Path = ReceivedDir + '\\HSIP Contract and Obligation Data.xlsx'
WPPS_Pr_Path = ReceivedDir + '\\HSIP Project Programmed Data.xlsx'
SPF_Data     = ReceivedDir  + '\\SPF Coefficients.xlsx'

2019-02-07 10:00:43


In [44]:
print(strftime("%Y-%m-%d %H:%M:%S"))
#Cont_DF = pd.concat([hsmpy3.common.FCtoDF(fc,readGeometry=True) for fc in [HSIP_Int,HSIP_Seg]])
Cont_DF = pd.read_excel(DocumentsDir + '\\Cont_DF.xlsx',index_col=0)
HSIP_DF = pd.read_excel(DocumentsDir + '\\HSIP_DF.xlsx')
HSIP_DF = HSIP_DF.reindex(sorted(HSIP_DF.columns), axis=1)
HSIP_Joined = pd.read_excel(DocumentsDir + '\\HSIP_Joined.xlsx')
HSIP_Joined.index = HSIP_Joined.HSIPID
HSIP_DF.index = HSIP_DF['HSIP ID']
for f in ['Problem Description','Location Description','Approved Scope']:
    HSIP_Joined[f] = HSIP_DF[f].loc[HSIP_Joined.index]
Seg_DF1 = hsmpy3.common.FCtoDF(Const_Seg,readGeometry=True)
Int_DF1 = hsmpy3.common.FCtoDF(Const_Int,readGeometry=True)
Cont_DF.loc[list(set(Seg_DF1.ContNum)),'Type'] = 'Seg'
Cont_DF.loc[list(set(Int_DF1.ContNum)),'Type'] = 'Int'
Pay_DF = pd.read_csv(ReceivedDir + '\\Contract_PayItems.csv')
Pay_DF.index = pd.MultiIndex.from_arrays([Pay_DF.ContNum,Pay_DF.PayItem])
def GetPLsCentroid(L):
    prts = []
    for p in L:
        for pr in p:
            prts.append(pr)
    g = arcpy.Geometry('LINE',arcpy.Array(prts),hsmpy3.common.NAD1983IL)
    p = arcpy.Geometry('POINT',g.centroid,hsmpy3.common.NAD1983IL)
    p = p.projectAs(hsmpy3.common.WGS1984)
    return('{},{}'.format(p.firstPoint.Y,p.firstPoint.X))
def GetPNsCentroid(L):
    pnts = []
    for p in L:
        pnts.append(p[0])
    g = arcpy.Geometry('multipoint',arcpy.Array(pnts),hsmpy3.common.NAD1983IL)
    p = arcpy.Geometry('POINT',g.centroid,hsmpy3.common.NAD1983IL)
    p = p.projectAs(hsmpy3.common.WGS1984)
    return('{},{}'.format(p.firstPoint.Y,p.firstPoint.X))
s1 = Seg_DF1.groupby('ContNum')['Shape'].aggregate(GetPLsCentroid)
s2 = Int_DF1.groupby('ContNum')['Shape'].aggregate(GetPNsCentroid)
s = pd.DataFrame(pd.concat([s1,s2]))
s['ContNum'] = s.index
df = pd.DataFrame(s.groupby('ContNum')['Shape'].agg({'Shape':'first'}))
df['Link'] = df.Shape.apply(lambda x:'https://www.google.com/maps?q=loc:{}&t=k'.format(x))
Cont_DF['Link'] = df.Link.loc[Cont_DF.index]

2019-02-01 08:15:53


In [4]:
BC_DF = pd.read_excel(r'\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO19\HSIP_Tracking_Tool\9.QC_Milhouse\QC_Contracts_FromKyle.xlsx')
CMF_DF = pd.read_excel(r'\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO19\HSIP_Tracking_Tool\9.QC_Milhouse\ServiceLife.xlsx',skiprows=3)
BC_DF = BC_DF.rename(columns={'Major Countermeasure (If there was no BC Summary Sheet)':'CounterMeasure'})

In [15]:
BC_DF[['ApplicationBC',
 'Has BC Summary Sheet in the attachments? (Yes,No)',
 'CMF (From BC Summary Sheet)',
 'Total Cost for each Countermeasure (From BC Summary Sheet)',
 'CounterMeasure',
 'Location (for multiple BC Summary Sheets)',
 'Additional Notes']].loc['60A25']

,ApplicationBC,"Has BC Summary Sheet in the attachments? (Yes,No)",CMF (From BC Summary Sheet),Total Cost for each Countermeasure (From BC Summary Sheet),CounterMeasure,Location (for multiple BC Summary Sheets),Additional Notes
60A25,1.44,No,NaN,NaN,1.4.1.I1.1 - Signalization - Install Traffic Signals,NaN,NaN


In [5]:
print(strftime("%Y-%m-%d %H:%M:%S"))
Trt_DF = pd.read_excel(r'\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO19\HSIP_Tracking_Tool\11.Plans_DataCollection\PlansDataCollection_12202018_Treatment.xlsx',index_col=[0,1])
Trt_DF = Trt_DF[['Treatment']]
Trt_DF['ContNum']  = Trt_DF.index.to_series().apply(lambda x:x[0])
Trt_DF =  pd.DataFrame(Trt_DF.groupby('ContNum')['Treatment'].first())
Trt_DF = Trt_DF[~pd.isnull(Trt_DF.Treatment)]
Trt_DF.Treatment = Trt_DF.Treatment.apply(lambda x:x.lower())
Cont_DF['Treatment'] = Trt_DF.Treatment.loc[Cont_DF.index]

2019-01-31 08:56:05


In [52]:
l = T_DF[(T_DF.T2==t) | (T_DF.T3==t)]['ContNum'].tolist()
l=['60I36']
pd.set_option('max_colwidth',200)
costcutoff = 0.999
i = l[0]
def LinkFromPoint(p):
    if p.type=='polyline':
        p = arcpy.Geometry('POINT',p.labelPoint,hsmpy3.common.NAD1983IL)
        p = p.projectAs(hsmpy3.common.WGS1984)
        s = '{},{}'.format(p.firstPoint.Y,p.firstPoint.X)
        s = 'https://www.google.com/maps?q=loc:{}&t=k'.format(s)
        return(s)
    else:
        p = p.projectAs(hsmpy3.common.WGS1984)
        s = '{},{}'.format(p.firstPoint.Y,p.firstPoint.X)
        s = 'https://www.google.com/maps?q=loc:{}&t=k'.format(s)
        return(s)
try:
    sL = []
    for s in Seg_DF1[Seg_DF1.ContNum==i]['Shape']:
        sL.append(s)
    for s in Int_DF1[Int_DF1.ContNum==i]['Shape']:
        sL.append(s)
    df = pd.DataFrame(columns=['Shape'])
    df['Shape'] = sL
    df['Type'] = df.Shape.apply(lambda x:x.type)
    df.loc[df.Type=='polyline','Length'] = df.loc[df.Type=='polyline','Shape'].apply(lambda x:x.length/5280.0)
    df['Link'] = df.Shape.apply(LinkFromPoint)
    for i,r in df.iterrows():
        print('Type: {}, Length: {}, Link: {}'.format(r.Type,r.Length,r.Link))
except:
    pass
pdesc = []
for i in l:
    #print('\n{}: {}'.format(i,Cont_DF.Link.loc[i]))
    print('Completed: {}, Total: ${:,.0f}, HSIP: ${:,.0f} ({:.2f} %)'.format(Cont_DF.F_CompDate.loc[i].strftime('%Y-%b'),Cont_DF.CompletionAmount.loc[i],Cont_DF.HSIPAmount.loc[i],Cont_DF.HSIPAmount.loc[i]/Cont_DF.CompletionAmount.loc[i]*100))
    print(Cont_DF.loc[i,'Impr_WPPS'])
    display(T_DF[T_DF.ContNum==i])
    try:
        display(BC_DF[['CounterMeasure']].loc[i])
    except:
        pass
    for h in Cont_DF.loc[i,'HSIPIDs'].split(';'):
        pdesc.append(HSIP_Joined.loc[int(h)]['Problem Description'])
        for c in ['Problem Description','Location Description','Approved Scope','AllSelectedImprovements','ProposedImprovements-Eng']:
            try:
                print('\n{}-{}-{}: {}'.format(i,h,c,HSIP_Joined.loc[int(h)][c].replace('\n',' ')))
            except:
                pass
    display(Pay_DF.loc[i,['PAY_ITEM_CODE','Unit','COMPLETED_QTY','COMPLETED_AMT']][Pay_DF.loc[i,'COMPLETED_AMT'].cumsum()<Pay_DF.loc[i,'COMPLETED_AMT'].sum()*costcutoff])
cns = []
allcns = list(set(Seg_DF1.ContNum.tolist()+Int_DF1.ContNum.tolist()))
for p in pdesc:
    for cn in HSIP_Joined[HSIP_Joined['Problem Description'] == p]['ContNums']:
        cns.extend([i for i in cn.split(';') if not i in l and i in allcns])
if len(cns)>0:
    for i in cns:
        print('\n{}: {}'.format(i,Cont_DF.Link.loc[i]))
        print('Completed: {}, Total: ${:,.0f}, HSIP: ${:,.0f} (%{:.1f})'.format(Cont_DF.F_CompDate.loc[i].strftime('%Y-%b'),Cont_DF.CompletionAmount.loc[i],Cont_DF.HSIPAmount.loc[i],Cont_DF.HSIPAmount.loc[i]/Cont_DF.CompletionAmount.loc[i]))
        print('WPPS Improvements: ' + Cont_DF.loc[i,'Impr_WPPS'])
        display(T_DF[T_DF.ContNum==i])
        display(BC_DF[['CounterMeasure']].loc[i])
        for h in Cont_DF.loc[i,'HSIPIDs'].split(';'):
            pdesc.append(HSIP_Joined.loc[int(h)]['Problem Description'])
            for c in ['Problem Description','Location Description','Approved Scope']:
                try:
                    print('\n{}-{}-{}: {}'.format(i,h,c,HSIP_Joined.loc[int(h)][c].replace('\n',' ')))
                except:
                    pass
    display(Pay_DF.loc[i,['PAY_ITEM_CODE','Unit','COMPLETED_QTY','COMPLETED_AMT']][Pay_DF.loc[i,'COMPLETED_AMT'].cumsum()<Pay_DF.loc[i,'COMPLETED_AMT'].sum()*costcutoff])

Type: polyline, Length: 3.2320382142064346, Link: https://www.google.com/maps?q=loc:42.1231611195303,-88.21320945714378&t=k
Type: polyline, Length: 1.4663530548226111, Link: https://www.google.com/maps?q=loc:42.14443738462863,-88.24711977478016&t=k
Type: point, Length: nan, Link: https://www.google.com/maps?q=loc:42.15124635641632,-88.25766745201967&t=k
Type: point, Length: nan, Link: https://www.google.com/maps?q=loc:42.11433999959266,-88.18319000046934&t=k
Type: point, Length: nan, Link: https://www.google.com/maps?q=loc:42.114064999806224,-88.17753000138602&t=k
Type: point, Length: nan, Link: https://www.google.com/maps?q=loc:42.110919999313715,-88.18073000132757&t=k
Type: point, Length: nan, Link: https://www.google.com/maps?q=loc:42.110339999425115,-88.17064000076185&t=k
Completed: 2010-Oct, Total: $747,296, HSIP: $600,475 (80.35 %)
FLASHING LIGHT;PAVEMENT MARKING;TRAF SIGNAL MODERNIZATION


,ContNum,T1,T2,T3
373,60I36,int,advance warning flashers,dynamic flashing beacons
374,60I36,int,traffic signal improvements,add signal head per lane
375,60I36,seg,guardrail,guardrail
376,60I36,seg,pavement marking,restripe the markings
377,60I36,seg,rumble strip,centerline and shoulder


CounterMeasure    3.4.3.I1.1 - Signalization - Flashing Beacon
Name: 60I36, dtype: object


60I36-201410257-Problem Description: This segment has experienced numerous severe head-on and sideswipe opposite direction collisions, resulting in 3 fatals and 7 A-injury crashes.  An additional 4 A-injury crashes were the result of fixed object collisions.

60I36-201410257-Location Description: IL Route 62 is generally a two lane undivided roadway with one lane in each direction and signalized intersections at IL Route 25 and IL Route 68.  This segment has unsignalized intersections at Autumn Trail East, Autumn Trail West, Helm Road, Bateman Road, Springwood Lane, and Old Sutton Road. Left turn channelization is provided at the intersections of Helm Road (westbound) and Bateman Road (eastbound and westbound). There are 2 equestrian trail crossings located between the intersections of Springwood Lane and Bateman Road. IL Route 62 is a 5-lane roadway just east of IL Route 25 and west of IL Route 68 before tapering down to its characteristic 2-lane cross section.  Between Old Sutton Ro

,PAY_ITEM_CODE,Unit,COMPLETED_QTY,COMPLETED_AMT
PayItem,,,,
40603340,"HOT-MIX ASPHALT SURFACE COURSE, MIX ""D"", N70",TON,1990.51,161231.31
63100167,"TRAFFIC BARRIER TERMINAL, TYPE 1 (SPECIAL) TANGENT",EACH,18.00,44658.00
44000157,"HOT-MIX ASPHALT SURFACE REMOVAL, 2""",Q YD,18498.00,41620.50
48203029,"HOT-MIX ASPHALT SHOULDERS, 8""",Q YD,897.70,40396.50
67100100,MOBILIZATION,SUM,1.00,40000.00
X9101201,NaN,NaN,96458.00,39547.78
X9100700,NaN,NaN,703.50,29019.38
X8620020,UNINTER POWER SUPPLY,EACH,5.00,24825.00
48102100,"AGGREGATE WEDGE SHOULDER, TYPE B",TON,425.24,20411.52


In [8]:
t = 'sign improvement'
T_DF = pd.read_excel(r'\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO19\HSIP_Tracking_Tool\11.Plans_DataCollection\Treatment_Data_Collection_20190201.xlsx')
T_DF = T_DF[~pd.isnull(T_DF.ContNum)]
T_DF = T_DF.fillna('')
T_DF.ContNum = T_DF.ContNum.astype(str)
#for c in ['T1','T2','T3']:
#    T_DF[c] = T_DF[c].apply(lambda x:x.lstrip().rstrip().lower())
#display(HTML(T_DF[(T_DF.T2==t) | (T_DF.T3==t)].to_html()))
HTML(pd.DataFrame(T_DF.groupby(['T2','T3'])['ContNum'].agg(lambda x:len(set(x))).sort_values(ascending=False)).to_html())

,,ContNum
T2,T3,
Shoulder Improvement,Add Paved Shoulder,83
Rumble Strip,Shoulder Rumbles,77
Traffic Signal Improvements,"Add/Upgrade Signal Head/Mast Arm (LED, Backplate, per Lane)",75
Guardrail\Attenuators,Guardrail,68
Turn Lane Improvements,Add Left Turn Lanes,49
Pavement Marking,Edgeline/Centerline Markings,33
Traffic Signal Improvements,Add/Improve Pedestrian Controls,33
Lighting,Add/Improve Street Lighting,30
Sign Improvement,Advance Intersection Signs (with Flashers),27


In [15]:
g = T_DF.groupby('ContNum')
g.T3.aggregate(lambda x:';'.join(x)).value_counts().to_csv('T_G.csv')

In [36]:
tdf = pd.Series()
for t in set(T_DF.T3):
    tdf.loc[t] = '; '.join([':'.join([i,str(r)]) for i,r in T_DF[T_DF.ContNum.isin(list(set(T_DF[T_DF.T3==t]['ContNum'])))]['T3'].value_counts().iteritems() if i!=t])
tdf.to_csv('T_G.csv')

In [49]:
l = T_DF.T3.value_counts().index.tolist()
tdf = pd.DataFrame(columns=l,index=l)
for t1 in l:
    for t2 in l:
        tdf.loc[t1,t2] = len(T_DF[T_DF.ContNum.isin(list(T_DF[T_DF.T3==t1]['ContNum']))][T_DF.T3==t2]['ContNum'])
tdf.to_csv('T2.csv')

C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to mat

C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to mat

C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to mat

C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to mat

C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to mat

...

C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to mat

C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to mat

C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to mat

C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to mat

C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to mat

In [48]:
T_DF.T3.value_counts().index.tolist()

['Add Paved Shoulder',
 'Shoulder Rumbles',
 'Add/Upgrade Signal Head/Mast Arm (LED, Backplate, per Lane)',
 'Guardrail',
 'Add Left Turn Lanes',
 'Add/Improve Pedestrian Controls',
 'Edgeline/Centerline Markings',
 'Add/Improve Street Lighting',
 'Advance Intersection Signs (with Flashers)',
 'Impact Attenuators',
 'Chevron on Horizontal Curves',
 'Converting Stop Controlled to Signal',
 'Cable Median Barrier',
 'Raised Reflective Edgline Marking',
 'WWD Improvements: Signing and Striping',
 'Convert Protecetd/Permissive to Protected Only',
 'Centerline and Shoulder Rumbles',
 'Resurfacing Alone',
 'Upgrade Signs to conform with MUTCD',
 'Convert Permissive to Protectetd/Permissive',
 'DeSlick',
 'Increase Left Turn Offset',
 'Add Right Turn Lanes',
 'Tree/Fixed Object Cleaning',
 'Post Mounted Delineators on Curves',
 'Flatten Roadside Slopes',
 'Improve Skew Angle',
 'Centerline Rumbles',
 'Flatten Horizontal Curve',
 'Concerete Median Barrier',
 'Arrows and Letters',
 'Convert Mino

In [34]:
t = 'Flatten Roadside Slopes'


'Cable Median Barrier:5; Add Paved Shoulder:1; Raised Reflective Edgline Marking:1'

In [29]:
CMF_DF = pd.read_excel(r'\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO19\HSIP_Tracking_Tool\9.QC_Milhouse\ServiceLife.xlsx',skiprows=3)
df = CMF_DF[~pd.isnull(CMF_DF.CMF)]
df = df[df.CMF!=0]
df.loc[613,'COUNTERMEASURES'] = '3.2.31.AL.1 - Pavement - Improve angle of channelized right turn lane'
df['H1'] = CMF_DF[['COUNTERMEASURES','CMF']][~pd.isnull(CMF_DF.CMF)]['COUNTERMEASURES'].apply(lambda x:x.split(' - ')[0].split('.')[0]).astype(int)
df['H2'] = CMF_DF[['COUNTERMEASURES','CMF']][~pd.isnull(CMF_DF.CMF)]['COUNTERMEASURES'].apply(lambda x:x.split(' - ')[0].split('.')[1]).astype(int)

df1 = CMF_DF[pd.isnull(CMF_DF.CMF) & (~pd.isnull(CMF_DF.COUNTERMEASURES))]
df1 = df1[df1['COUNTERMEASURES'].apply(lambda x:x.split('.')[1][0]=='0')][['COUNTERMEASURES']]
df1['H1T'] = df1.COUNTERMEASURES.apply(lambda x:x[4:])
df1['H1'] = df1.COUNTERMEASURES.apply(lambda x:x[0])
df1 = df1[['H1','H1T']]
df1.index = df1.H1.astype(int)
df1.H1T = ['.'.join([str(i),x.H1T]) for i,x in df1.iterrows()]
df['H1T'] = list(df1.H1T.loc[list(df.H1.astype(int))])

df1 = CMF_DF[pd.isnull(CMF_DF.CMF) & (~pd.isnull(CMF_DF.COUNTERMEASURES))]
df1 = df1[df1['COUNTERMEASURES'].apply(lambda x:x.split('.')[1][0]!='0')][['COUNTERMEASURES']].iloc[:-1]
df1['H2T'] = df1.COUNTERMEASURES.apply(lambda x:x[2:]).apply(lambda x:'.'.join([x.split(' ')[0],' '.join(x.split(' ')[1:]).lstrip()]))
df1['H2'] = df1.H2T.apply(lambda x:x[0]).astype(int)
df1['H1'] = df1.COUNTERMEASURES.apply(lambda x:x[0]).astype(int)
df1.index = pd.MultiIndex.from_arrays([df1.H1,df1.H2])
df['H2T'] = list(df1.H2T.loc[pd.MultiIndex.from_arrays([df.H1,df.H2])])

df['H3T'] = df.COUNTERMEASURES.apply(lambda x:' - '.join(x.split(' - ')[2:]))
df['H3'] = df.COUNTERMEASURES.apply(lambda x:x.split('.')[2]).astype(int)
df['H3T'] = ['.'.join([str(i),x]) for i,x in zip(df.H3,df.H3T)]
#df.index = pd.MultiIndex.from_arrays([df.H1T,df.H2T,df.H3T])
#d.DataFrame(df.groupby(['H1T','H2T','H3T']).size())
#d.DataFrame(df.groupby(['H3T']).size())
F = ['H1T','H2T','H3T','COUNTERMEASURES', 'Service Life', 'CMF', 
 'Crash Type Affected by Countermeasures', 'All', 'AG', 'AN', 'FO', 'HO', 'LT', 'OtherNC', 'OtherO', 'OVT', 'PD', 'PDC', 'PKV', 'RE', 'RT', 'SSD', 'SOD', 'T', 'TR', 'NGT', 'ROR', 'WP', 
 'Setting Type', 'Facility Type', 'AADTMajor', 'AADTMinor', 'UserDefined1', 'UserDefined2', 
 'Standard Error', 'Source', 
 'HSM Applicability Flag', 'AADT Flag', 'Special Case Flag', 'AADT MajorMin', 'AADT MajorMax', 'AADT MinorMin', 'AADT MinorMax', 
 'CMF "All" Adjustment Flag', 'Adjusted CMF', 'CMF Project Type', 'CMF Project System', 
 'All.1', 'AG.1', 'AN.1', 'FO.1', 'HO.1', 'LT.1', 'OtherNC.1', 'OtherO.1', 'OVT.1', 'PD.1', 'PDC.1', 'PKV.1', 'RE.1', 'RT.1', 'SSD.1', 'SOD.1', 'T.1', 'TR.1', 'NGT.1', 'ROR.1', 'WP.1', 'CMF Applicability Flag']
df.index = pd.MultiIndex.from_arrays([df.H1T,df.H2T,df.H3T])
df = df[F]
df.to_excel('BC_Tool_CMFs.xlsx')

In [65]:
df = pd.DataFrame(Pay_DF.groupby('ContNum')['COMPLETED_AMT'].aggregate(sum))
idx = Cont_DF[Cont_DF.CompletionAmount==0].index
Cont_DF.CompletionAmount.loc[idx] = df.COMPLETED_AMT.loc[idx].fillna(0)

C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:1: FutureWarning: 'ContNum' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  """Entry point for launching an IPython kernel.
C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:3: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexi

In [76]:
l = []
for i,r in Trt_DF.iterrows():
    s = r.item().lower().replace(';',',').replace('.',',')
    l1 = s.split(',')
    for l2 in l1:
        l.append((i,l2))
df = pd.DataFrame(columns=['ContNum','Type','Treatment'],index = pd.MultiIndex.from_tuples(l))
df.ContNum = df.index.to_series().apply(lambda x:x[0])
df.Treatment = df.index.to_series().apply(lambda x:x[1])
idx = df.index
df.index = df.ContNum
df.Type = Cont_DF.Type.loc[df.index]
df.index = idx
df.to_excel('Treatment_Data_Collection.xlsx',index=False)

C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:12: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  if sys.path[0] == '':


In [43]:
df = pd.read_excel(r'\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO19\HSIP_Tracking_Tool\11.Plans_DataCollection\PlansDataCollection_12202018_Treatment.xlsx',index_col=[0,1])
df = df[df.Action=='Geocoded'][['Treatment']]
df['ContNum']  = df.index.to_series().apply(lambda x:x[0])
df =  pd.DataFrame(df.groupby('ContNum')['Treatment'].first())
l = []
for s in list(set(df.Treatment.value_counts().index)):
    s =s.lower().replace(';',',').replace('.',',')
    l1 = s.split(',')
    l2 = [i.split(';') for i in l1]
    l1 = []
    for l3 in l2:
        l1.extend(l3)
    l.extend(l1)
l = list(set(l))
l = list(set([i.lstrip().rstrip() for i in l]))
l.sort()
l

C:\Users\mr068144\AppData\Roaming\Python\Python35\site-packages\ipykernel_launcher.py:4: FutureWarning: 'ContNum' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  after removing the cwd from sys.path.


['',
 'add/improve left turn phase',
 'add/improve turn lanes',
 'advance warning flashers',
 'cable median barrier',
 'centerline rumble strip',
 'channelization',
 'chevrons and/or delineators',
 'curve realignment',
 'deslick',
 'guardrail',
 'impact attenuators',
 'improve lighting',
 'improvement/realignment/reconstruction',
 'install traffic signal',
 'median barrier',
 'pavement marking',
 'pedestrian controlls improvements',
 'raised pavement marking',
 'resurfacing',
 'roadside improvement',
 'roundabout',
 'rumble strip',
 'shoulder improvement/widening',
 'sight distance imrpovement',
 'sign improvement',
 'stop controlled intersection improvement',
 'traffic signal improvements',
 'traffic/median barriers',
 'widening and/or resurfacing']